# Attendance System Streamlit

---



1. Create folder named "database"
2. Place images in it. name them in string
3. this code is for CPU Colab
4. 12/Dec/2024 Danish Ammar

In [ ]:
# Install required packages (run this in a separate cell first)
!pip install streamlit deepface opencv-python pandas numpy

In [ ]:
%%writefile app.py
import streamlit as st
import cv2
from deepface import DeepFace
import numpy as np
import pandas as pd
from datetime import datetime
import os
from PIL import Image
import io



# Supported image formats
IMAGE_FORMATS = ('.jpg', '.jpeg')

# Create directories if they don't exist
if not os.path.exists("database"):
    os.makedirs("database")
if not os.path.exists("attendance"):
    os.makedirs("attendance")

def get_registered_users():
    users = {}
    for file in os.listdir("database"):
        if file.lower().endswith(IMAGE_FORMATS):
            name = os.path.splitext(file)[0]
            file_path = os.path.join("database", file)
            users[name] = file_path
    return users

def mark_attendance(name):
    date = datetime.now().strftime("%Y-%m-%d")
    time = datetime.now().strftime("%H:%M:%S")

    df = pd.DataFrame(columns=['Name', 'Date', 'Time'])
    attendance_file = f"attendance/{date}.csv"

    if os.path.exists(attendance_file):
        df = pd.read_csv(attendance_file)

    if not df['Name'].str.contains(name).any():
        new_attendance = pd.DataFrame([[name, date, time]], columns=['Name', 'Date', 'Time'])
        df = pd.concat([df, new_attendance], ignore_index=True)
        df.to_csv(attendance_file, index=False)
        return True, f"Attendance marked for {name}"
    else:
        return False, f"Attendance already marked for {name}"

def recognize_face(frame):
    registered_users = get_registered_users()

    for name, img_path in registered_users.items():
        try:
            db_img = cv2.imread(img_path)
            if db_img is None:
                continue

            result = DeepFace.verify(frame, db_img, enforce_detection=False)

            if result["verified"]:
                return name

        except Exception:
            continue

    return None

def view_attendance():
    date = datetime.now().strftime("%Y-%m-%d")
    attendance_file = f"attendance/{date}.csv"

    if os.path.exists(attendance_file):
        return pd.read_csv(attendance_file)
    return None

def main():
    st.title("Face Recognition Attendance System")

    menu = ["Take Attendance", "View Attendance", "Register New User"]
    choice = st.sidebar.selectbox("Menu", menu)

    if choice == "Take Attendance":
        st.subheader("Take Attendance")

        img_file = st.camera_input("Take a photo")

        if img_file is not None:
            # Convert the image to OpenCV format
            bytes_data = img_file.getvalue()
            img_array = np.frombuffer(bytes_data, np.uint8)
            frame = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

            # Recognize face
            name = recognize_face(frame)

            if name:
                success, message = mark_attendance(name)
                if success:
                    st.success(message)
                else:
                    st.info(message)
            else:
                st.error("No registered face found in the image!")

    elif choice == "View Attendance":
        st.subheader("Today's Attendance")

        df = view_attendance()
        if df is not None:
            st.dataframe(df)
        else:
            st.info("No attendance records for today")

    elif choice == "Register New User":
        st.subheader("Register New User")

        name = st.text_input("Enter name")
        img_file = st.camera_input("Take a photo")

        if name and img_file:
            # Save the image to database
            bytes_data = img_file.getvalue()
            file_path = os.path.join("database", f"{name}.jpg")

            with open(file_path, "wb") as f:
                f.write(bytes_data)

            st.success(f"User {name} registered successfully!")

# For running in Colab, save this as app.py and run:
# !streamlit run app.py & npx localtunnel --port 8501

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
!pip install pyngrok

In [ ]:
import os
from threading import Thread
from pyngrok import ngrok

In [ ]:
from google.colab import userdata

# Retrieve the Ngrok API key securely
ngrok_api_key = userdata.get('NGROK_API')

if ngrok_api_key:
    # Authenticate Ngrok with the API key
    ngrok.set_auth_token(ngrok_api_key)
    print("Ngrok authenticated successfully!")
else:
    print("Ngrok API key not found. Please add it in the Colab settings.")


Ngrok authenticated successfully!


In [ ]:
# Add your ngrok token here
ngrok.set_auth_token(ngrok_api_key)

In [ ]:
def run_streamlit():
    # Change the port if 8501 is already in use or if you prefer another port
    os.system('streamlit run /content/app.py --server.port 8501')


In [ ]:
# Start a thread to run the Streamlit app
thread = Thread(target=run_streamlit)
thread.start()


In [ ]:

# Open a tunnel to the streamlit port 8501
public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)
print('Your Streamlit app is live at:', public_url)

Your Streamlit app is live at: NgrokTunnel: "https://d146-34-138-50-157.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
ngrok.kill()